# Plotting the price of assets against Elon Musk and Tesla Twitter activity 

## Why?

Over the last few months my news feed has been a constant wash of articles about Blockchain, stock markets (GME) and Elon Musk. The first and the last of these topics have been newsworthy regularly over the last few years, meanwhile the GME shortsqueeze appeared overnight.

There's no doubt that these topics are interconnected. Homebound US citizens with stimulus checks and too much time seem to be the main participants in these three hot topics.

Whilst this has been going on I have found myself increasingly interested in data science, more specifically data visualisation, I was introduced to the field by a colleague and sites like ChartR (https://www.chartr.co/) have gone a long way in furthering that interest. Stories told through a simple vis are compelling. A picture (or chart) speak a thousand words.

To attempt to get into the field I have been taking an extended course on data science provided by IBM. I found this course good, but lacking applied projects..so here goes an attempt at applying some of what I have known.

## Why this post?

This isn't really written to garner an audience - more for me to catalogue my progress over what I hope will be a long while.

## The project

The plan is to look at the Bitcoin, Dogecoin and GME prices along side Mr Musks Twitter activity to see if I can see any trends.

It should be noted this is not a rigerous quantitative approach - more a high level 'lets see what I can achieve in a day or so and check what I've learned' approach. If you're after the former then have a look at this: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3778844

In [10]:
#To start import pandas to enable easier data manipulation, Plotly for eventual visualisation, yfinance to obtain our data

import pandas as pd

import plotly.express as px

import yfinance as yf

In [84]:
# Download the required data from yahoo finance using the appropriate tickers 
# Using start Dec 2020 to mid march (around the time of writing)
# Take daily prices on the assumption tweets are around that frequent

#BTC
BTC_Data = yf.download(tickers='BTC-USD', start = '2020-12-01', end = '2021-03-14', interval = '1d')

#It's worth noting not reseting the index caused a major headache as the 'date' column header was misaligned without it.
BTC_Data.reset_index(inplace=True)

#DOGE
DOGE_Data = yf.download(tickers='DOGE-USD', start = '2020-12-01', end = '2021-03-14', interval = '1d')

DOGE_Data.reset_index(inplace=True)

#GME
GME_Data = yf.download(tickers='GME', start = '2020-12-01', end = '2021-03-14', interval = '1d')

GME_Data.reset_index(inplace=True)

#I tried running the head of all three downnloads but Python will only show one so that will suffice to show its working and what we have returned

BTC_Data.head(5)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-12-01,17.110001,17.400000,15.760000,15.800000,15.800000,12666000
1,2020-12-02,15.700000,16.680000,15.380000,16.580000,16.580000,7883400
2,2020-12-03,16.480000,16.639999,15.870000,16.120001,16.120001,6295000
3,2020-12-04,16.299999,17.290001,16.260000,16.900000,16.900000,8972700
4,2020-12-07,17.000000,17.500000,16.219999,16.350000,16.350000,7386300


### Quick note on obtaining tweets

How would we obtain the tweet information? My initial thought process was:

- Scrape the tweets using the Twitter API 
- Create a table of the tweets with datestamp and text
- Filter by keyword e.g. BTC, Bitcoin etc.
- Plot the tweets using date stamp over the price graph 

this would likely involve appending prices to the table so that we can plot an effective 'x and y' coordinate of date and price and then use the text as a data label - that's the theory at least...

In scraping the tweets I came across Tweepy, which is a convinient pre-made package that enables you to scrape tweets without too much prior knowledge. I ended up using it and an article from Martin Beck: https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1 to achieve what I wanted. I won't claim I spent too much time examining what was going on in the next cell but I cared more that it worked, and really that's the beauty of the ds community - open knowledge sharing. It's what makes it approachable to a complete novice like myself. The cell below is far better explained by the article above so I won't go into much detail.



In [85]:
# Import what is required
import tweepy
import time

# Credentials to enable access to the Twitter API

consumer_key = "qSlXdQBHiP3bd45cCOs9Sq0FO"
consumer_secret = "I3W8CBFDkhgwCFp6VExQj3FhTHEkcuMXMAlgrj88MS7WaEQxP3"
access_token = "604056369-pGIpZHnepCzzs4yXdF9clWV10UxLy1mX7kEhW4ir"
access_token_secret = "0pHtZQgE2QHc4tP9JyU5VCKuTjjHdh38Qxzpz8MUlWZex"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

#Here the code defines a function that we can call later on to get the data we need

tweets = []

def username_tweets_to_csv(username,count):
    try:      
        # Creation of query method using parameters
        tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)

        # Pulling information from tweets iterable object
        tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]

        # Creation of dataframe from tweets list
        # Add or remove columns as you remove tweet information
        tweets_df = pd.DataFrame(tweets_list,columns=['Datetime', 'Tweet Id', 'Text'])

        # Converting dataframe to CSV 
        tweets_df.to_csv('{}-tweets.csv'.format(username), sep=',', index = False)

    except BaseException as e:
          print('failed on_status,',str(e))
          time.sleep(3)
        
# Input username to scrape tweets and name csv file
# Max recent tweets pulls x amount of most recent tweets from that user
username = 'elonmusk'
count = 100

# Calling function to turn username's past X amount of tweets into a CSV file
username_tweets_to_csv(username, count)

,Datetime,Tweet Id,Text
0,2021-03-16 03:50:21,1371670142892044291,RT @mayascade: stevie wonder performing “super...
1,2021-03-16 01:08:13,1371629341508050945,RT @elonmusk: I’m selling this song about NFTs...
2,2021-03-16 00:07:29,1371614057338376193,"Congratulations, ⁦@Beyonce⁩!\n\n https://t.co/..."
3,2021-03-15 23:08:50,1371599299864367107,💯 https://t.co/vL7Q9VBeRF
4,2021-03-15 19:52:40,1371549932000313351,"RT @signalapp: As a nonprofit organization, we..."


In [ ]:
#import the created CSV files and create a pandas data frame

csv_path = '/Users/jamesdawson/Desktop/Data analysis /Python/jack-tweets.csv'

df = pd.read_csv(csv_path)

df.head(5)

In [45]:
#Filter out anything that we don't feel is relevant from the tweets
#We need to create three data frames here so we can plot each respective one against a different plot of price

df[df['Text'].str.contains("bitcoin")]

,Datetime,Tweet Id,Text
11,2021-03-12 19:11:30,1370452409957384198,RT @hello_bitcoin: Many critics think bitcoin ...
39,2021-02-25 23:32:42,1365082324900012036,RT @bitcoinzay: Starting in less than an hour!...
43,2021-02-23 17:13:31,1364262123417726976,@bitcoinbrink 🙏🏼
51,2021-02-10 16:53:52,1359546136785129473,RT @moneyball: Great overview of what the bitc...
53,2021-02-06 21:46:27,1358170216136511489,@_jonasschnelli_ @gonzoucab # here's how I did...
56,2021-02-05 19:47:18,1357777841098682373,RT @bitcoinzay: The Black #Bitcoin Billionaire...
59,2021-02-05 07:15:55,1357588751757627393,Running #bitcoin https://t.co/W51ga3yrKb
65,2021-02-01 22:13:04,1356364974075990016,RT @MuunWallet: We're excited to share Muun's ...
91,2021-01-13 04:27:06,1349211345615421440,@bitcoinzay 🙏🏼


In [ ]:
#append the price to the line based on datetime - How easy this would be if I was using index match in excel..



### Comment on data collection

Though it would be nice to press run and we get everything we need pulled down without too much manual entry, at the end of the day you need consider whether it's worth it to automate some things. There's certain bits of news that I have in the back of my mind that aren't tweets but I do feel would compliment the analysis - to do this you can create manual annotations and place them onto the graph by eye.

In [78]:
fig = px.line(BTC_Data, x = "Date", y = "Open", title = 'BTC Price over time, monthly')

#This hit the news straight away
fig.add_annotation(x= "2021-01-29 00:00:00.00000", y=34111,
            text="Elon Musk changes his bio to #bitcoin",
            showarrow=True,
            arrowhead=1)


#A friend showed me this 
fig.add_annotation(x= "2021-02-08 10:00:00.00000", y=46196,
            text="Tesla's 10-K return shows $1.5bn of bitcoin holdings",
            showarrow=True,
            arrowhead=1)

#As a holder of eth I remember this one quite well..
fig.add_annotation(x= "2021-02-20 00:00:00.00000", y=55800,
            text="Elon Musk tweets 'BTC & ETH do seem high lol'",
            showarrow=True,
            arrowhead=1)


fig.show()